In [20]:
from langchain_community.document_loaders import PyPDFLoader as pdfldr
docs = pdfldr('psychology_book.pdf').load()
print(type(docs))

#print(len(docs), docs[0].page_content[:300])


<class 'list'>


In [21]:
docs[150].metadata

{'producer': 'Prince 14.2 (www.princexml.com)',
 'creator': 'PyPDF',
 'creationdate': '2022-02-24T09:25:50-06:00',
 'moddate': '2022-03-01T11:18:04-06:00',
 'title': 'Psychology 2e',
 'source': 'psychology_book.pdf',
 'total_pages': 753,
 'page': 150,
 'page_label': '151'}

In [22]:
print(type(docs[5]))

<class 'langchain_core.documents.base.Document'>


In [23]:
docs[6].page_content


'CONTENTS\nPreface 1\nCHAPTER 1\nIntroduction to Psychology 7\nIntroduction 7\n1.1 What Is Psychology? 8\n1.2 History of Psychology 9\n1.3 Contemporary Psychology 18\n1.4 Careers in Psychology 26\nKey Terms 30\nSummary 30\nReview Questions 32\nCritical Thinking Questions 34\nPersonal Application Questions 34\nCHAPTER 2\nPsychological Research 35\nIntroduction 35\n2.1 Why Is Research Important? 36\n2.2 Approaches to Research 41\n2.3 Analyzing Findings 48\n2.4 Ethics 59\nKey Terms 63\nSummary 64\nReview Questions 66\nCritical Thinking Questions 69\nPersonal Application Questions 70\nCHAPTER 3\nBiopsychology 71\nIntroduction 71\n3.1 Human Genetics 72\n3.2 Cells of the Nervous System 78\n3.3 Parts of the Nervous System 84\n3.4 The Brain and Spinal Cord 86\n3.5 The Endocrine System 97\nKey Terms 100\nSummary 102\nReview Questions 103\nCritical Thinking Questions 106\nPersonal Application Questions 106\nCHAPTER 4\nStates of Consciousness 109\nIntroduction 109\n4.1 What Is Consciousness? 110'

In [24]:
page_lookup = {}

for d in docs:
    page_label = d.metadata.get("page_label")
    if page_label is not None:
        page_lookup[int(page_label)] = d


## Retriving only relevant information in the text book, which includes:
### Keeping and adding:
1. All numbered structured sections (e.g., 6.1, 6.2, 14.1, 14.2, etc.)
2. Adding chapter numbers to the metadata
### Excluding:
1. Introduction
2. Key Terms
3. Summary
4. Review Questions
5. Critical Thinking Questions
6. Personal Application Questions



In [25]:
section_pages = {

    "1.1": 20,
    "1.2": 21,
    "1.3": 30,
    "1.4": "38-41",

    "2.1": 48,
    "2.2": 53,
    "2.3": 60,
    "2.4": "71-74",

    "3.1": 84,
    "3.2": 90,
    "3.3": 96,
    "3.4": 98,
    "3.5": "109-111",

    "4.1": 122,
    "4.2": 126,
    "4.3": 129,
    "4.4": 133,
    "4.5": 138,
    "4.6": "146-148",

    "5.1": 158,
    "5.2": 161,
    "5.3": 165,
    "5.4": 173,
    "5.5": 176,
    "5.6": "180-183",

    "6.1": 194,
    "6.2": 195,
    "6.3": 204,
    "6.4": "215-216",

    "7.1": 226,
    "7.2": 230,
    "7.3": 234,
    "7.4": 240,
    "7.5": 243,
    "7.6": "249-252",

    "8.1": 260,
    "8.2": 267,
    "8.3": 271,
    "8.4": "281-284",

    "9.1": 292,
    "9.2": 296,
    "9.3": 304,
    "9.4": "325-326",

    "10.1": 334,
    "10.2": 340,
    "10.3": 346,
    "10.4": "354-364",

    "11.1": 372,
    "11.2": 374,
    "11.3": 380,
    "11.4": 385,
    "11.5": 389,
    "11.6": 390,
    "11.7": 391,
    "11.8": 396,
    "11.9": "398-402",

    "12.1": 412,
    "12.2": 418,
    "12.3": 421,
    "12.4": 427,
    "12.5": 434,
    "12.6": 441,
    "12.7": "444-448",

    "13.1": 460,
    "13.2": 468,
    "13.3": 479,
    "13.4": "489-491",

    "14.1": 498,
    "14.2": 508,
    "14.3": 514,
    "14.4": 526,
    "14.5": "533-540",

    "15.1": 550,
    "15.2": 554,
    "15.3": 557,
    "15.4": 560,
    "15.5": 566,
    "15.6": 570,
    "15.7": 572,
    "15.8": 582,
    "15.9": 586,
    "15.10": 588,
    "15.11": "594-600",

    "16.1": 612,
    "16.2": 617,
    "16.3": 629,
    "16.4": 633,
    "16.5": "635-638",
}




In [26]:
section_headings = {

    "1.1": "1.1 What Is Psychology?",
    "1.2": "1.2 History of Psychology",
    "1.3": "1.3 Contemporary Psychology",
    "1.4": "1.4 Careers in Psychology",

    "2.1": "2.1 Why Is Research Important?",
    "2.2": "2.2 Approaches to Research",
    "2.3": "2.3 Analyzing Findings",
    "2.4": "2.4 Ethics",

    "3.1": "3.1 Human Genetics",
    "3.2": "3.2 Cells of the Nervous System",
    "3.3": "3.3 Parts of the Nervous System",
    "3.4": "3.4 The Brain and Spinal Cord",
    "3.5": "3.5 The Endocrine System",

    "4.1": "4.1 What Is Consciousness?",
    "4.2": "4.2 Sleep and Why We Sleep",
    "4.3": "4.3 Stages of Sleep",
    "4.4": "4.4 Sleep Problems and Disorders",
    "4.5": "4.5 Substance Use and Abuse",
    "4.6": "4.6 Other States of Consciousness",

    "5.1": "5.1 Sensation versus Perception",
    "5.2": "5.2 Waves and Wavelengths",
    "5.3": "5.3 Vision",
    "5.4": "5.4 Hearing",
    "5.5": "5.5 The Other Senses",
    "5.6": "5.6 Gestalt Principles of Perception",

    "6.1": "6.1 What Is Learning?",
    "6.2": "6.2 Classical Conditioning",
    "6.3": "6.3 Operant Conditioning",
    "6.4": "6.4 Observational Learning (Modeling)",

    "7.1": "7.1 What Is Cognition?",
    "7.2": "7.2 Language",
    "7.3": "7.3 Problem Solving",
    "7.4": "7.4 What Are Intelligence and Creativity?",
    "7.5": "7.5 Measures of Intelligence",
    "7.6": "7.6 The Source of Intelligence",

    "8.1": "8.1 How Memory Functions",
    "8.2": "8.2 Parts of the Brain Involved with Memory",
    "8.3": "8.3 Problems with Memory",
    "8.4": "8.4 Ways to Enhance Memory",

    "9.1": "9.1 What Is Lifespan Development?",
    "9.2": "9.2 Lifespan Theories",
    "9.3": "9.3 Stages of Development",
    "9.4": "9.4 Death and Dying",

    "10.1": "10.1 Motivation",
    "10.2": "10.2 Hunger and Eating",
    "10.3": "10.3 Sexual Behavior",
    "10.4": "10.4 Emotion",

    "11.1": "11.1 What Is Personality?",
    "11.2": "11.2 Freud and the Psychodynamic Perspective",
    "11.3": "11.3 Neo-Freudians: Adler, Erikson, Jung, and Horney",
    "11.4": "11.4 Learning Approaches",
    "11.5": "11.5 Humanistic Approaches",
    "11.6": "11.6 Biological Approaches",
    "11.7": "11.7 Trait Theorists",
    "11.8": "11.8 Cultural Understandings of Personality",
    "11.9": "11.9 Personality Assessment",

    "12.1": "12.1 What Is Social Psychology?",
    "12.2": "12.2 Self-presentation",
    "12.3": "12.3 Attitudes and Persuasion",
    "12.4": "12.4 Conformity, Compliance, and Obedience",
    "12.5": "12.5 Prejudice and Discrimination",
    "12.6": "12.6 Aggression",
    "12.7": "12.7 Prosocial Behavior",

    "13.1": "13.1 What Is Industrial and Organizational Psychology?",
    "13.2": "13.2 Industrial Psychology: Selecting and Evaluating Employees",
    "13.3": "13.3 Organizational Psychology: The Social Dimension of Work",
    "13.4": "13.4 Human Factors Psychology and Workplace Design",

    "14.1": "14.1 What Is Stress?",
    "14.2": "14.2 Stressors",
    "14.3": "14.3 Stress and Illness",
    "14.4": "14.4 Regulation of Stress",
    "14.5": "14.5 The Pursuit of Happiness",

    "15.1": "15.1 What Are Psychological Disorders?",
    "15.2": "15.2 Diagnosing and Classifying Psychological Disorders",
    "15.3": "15.3 Perspectives on Psychological Disorders",
    "15.4": "15.4 Anxiety Disorders",
    "15.5": "15.5 Obsessive-Compulsive and Related Disorders",
    "15.6": "15.6 Posttraumatic Stress Disorder",
    "15.7": "15.7 Mood and Related Disorders",
    "15.8": "15.8 Schizophrenia",
    "15.9": "15.9 Dissociative Disorders",
    "15.10": "15.10 Disorders in Childhood",
    "15.11": "15.11 Personality Disorders",

    "16.1": "16.1 Mental Health Treatment: Past and Present",
    "16.2": "16.2 Types of Treatment",
    "16.3": "16.3 Treatment Modalities",
    "16.4": "16.4 Substance-Related and Addictive Disorders: A Special Case",
    "16.5": "16.5 The Sociocultural Model and Therapy Utilization",
}


### PARENT LEVEL CHUNKING WHERE EVERY INDIVIDUAL CHUNK IS ONE OF THE ABOVE SECTIONS

In [27]:
from typing import Dict, List, Tuple, Optional, Union
from langchain_core.documents import Document

def page_label_int(d: Document) -> int:
    pl = d.metadata.get("page_label")
    if pl is None:
        return int(d.metadata.get("page", 0)) + 1
    return int(str(pl))

def normalize_for_search(s: str) -> str:
    # tolerate "1.1What" vs "1.1 What" etc.
    import re
    return re.sub(r"\s+", "", s).lower()

def find_heading_index(raw_text: str, heading: str) -> int:
    """
    Return character index in raw_text where the heading begins, or -1 if not found.
    Uses whitespace-insensitive matching.
    """
    if not raw_text or not heading:
        return -1

    norm_text = normalize_for_search(raw_text)
    norm_heading = normalize_for_search(heading)

    pos = norm_text.find(norm_heading)
    if pos == -1:
        return -1

    # Approximate mapping back to raw_text:
    # Find the section id token (e.g., "1.1") and validate heading near it.
    sec_id = heading.strip().split()[0]  # "1.1"
    raw_pos = raw_text.find(sec_id)
    if raw_pos == -1:
        return -1

    window = raw_text[max(0, raw_pos - 200): raw_pos + 1200]
    if normalize_for_search(window).find(norm_heading) != -1:
        return raw_pos

    return -1

def section_sort_key(sec_id: str) -> Tuple[int, int]:
    ch, sub = sec_id.split(".")
    return int(ch), int(sub)

def parse_section_value(v: Union[int, str]) -> Tuple[int, Optional[int]]:
    """
    If v is int -> (start, None)
    If v is "start-end" -> (start, end)
    """
    if isinstance(v, int):
        return v, None
    v = str(v).strip()
    if "-" in v:
        a, b = v.split("-", 1)
        return int(a.strip()), int(b.strip())
    return int(v), None

def build_page_lookup(docs: List[Document]) -> Dict[int, Document]:
    lookup = {}
    for d in docs:
        lookup[page_label_int(d)] = d
    return lookup

def extract_section_text(
    page_lookup: Dict[int, Document],
    sec_heading: str,
    start_page: int,
    next_heading: Optional[str],
    next_start_page: Optional[int],
    hard_end_page: Optional[int] = None,
) -> Tuple[str, List[int]]:
    """
    Pure jump logic (no scanning intermediate pages for next heading):

    1) On start_page: find sec_heading and start from there.
    2) Add full pages start_page+1 .. (next_start_page-1) (or .. hard_end_page if set).
    3) On next_start_page: cut everything before next_heading and include that as last part.
       (If hard_end_page is set and ends before next_start_page, we stop at hard_end_page.)
    """

    if start_page not in page_lookup:
        return "", []

    # Determine stopping boundary
    if hard_end_page is not None:
        stop_page = hard_end_page
    elif next_start_page is not None:
        stop_page = next_start_page
    else:
        raise ValueError("Need either hard_end_page or next_start_page to know where to stop.")

    collected_parts: List[str] = []
    pages_used: List[int] = []

    # 1) Start page: find current heading
    raw0 = page_lookup[start_page].page_content
    idx0 = find_heading_index(raw0, sec_heading)
    if idx0 == -1:
        # If you are 100% sure structure is correct, you can raise instead of returning empty
        return "", []

    raw0 = raw0[idx0:]
    pages_used.append(start_page)

    # If next section starts on the same page, cut immediately
    if next_heading:
        nxt0 = find_heading_index(raw0, next_heading)
        if nxt0 != -1:
            collected_parts.append(raw0[:nxt0].rstrip())
            return "\n".join(collected_parts).strip(), pages_used

    collected_parts.append(raw0.rstrip())

    # 2) Middle pages (full pages, no heading searches)
    mid_end = stop_page - 1 if (next_start_page is not None and stop_page == next_start_page) else stop_page
    for p in range(start_page + 1, mid_end + 1):
        d = page_lookup.get(p)
        if not d:
            continue
        collected_parts.append(d.page_content.rstrip())
        pages_used.append(p)

    # 3) Boundary page: next_start_page (only if stop_page is next_start_page and we have next_heading)
    if next_start_page is not None and stop_page == next_start_page and next_heading:
        dN = page_lookup.get(next_start_page)
        if dN:
            rawN = dN.page_content
            pages_used.append(next_start_page)

            nxt_idx = find_heading_index(rawN, next_heading)
            if nxt_idx != -1:
                collected_parts.append(rawN[:nxt_idx].rstrip())
            else:
                # If not found, include nothing (strict), or include whole page (lenient).
                # Strict choice:
                pass

    return "\n".join(collected_parts).strip(), pages_used


### parent chunks

In [28]:
from langchain_core.documents import Document

def parse_start_page(v):
    # v can be int or "start-end"
    if isinstance(v, int):
        return v
    return int(str(v).split("-")[0].strip())

def parse_end_page(v):
    # returns end page if "start-end", else None
    if isinstance(v, str) and "-" in v:
        return int(v.split("-")[1].strip())
    return None

def section_sort_key(sec_id: str):
    ch, sub = sec_id.split(".")
    return (int(ch), int(sub))

def build_section_documents_jump(section_pages, section_headings, page_lookup):
    """
    Returns a list of Document objects (one per section), ordered by section id.
    section_docs[0] -> 1.1, section_docs[1] -> 1.2, ...
    Uses extract_section_text_jump_only (pure jump logic).
    """

    sorted_sections = sorted(section_pages.keys(), key=section_sort_key)
    section_docs = []

    for i, sec_id in enumerate(sorted_sections):
        start_page = parse_start_page(section_pages[sec_id])
        hard_end_page = parse_end_page(section_pages[sec_id])  # only set for "range" sections

        # Next section boundary
        next_heading = None
        next_start_page = None
        if i + 1 < len(sorted_sections):
            next_sec = sorted_sections[i + 1]
            next_heading = section_headings[next_sec]
            next_start_page = parse_start_page(section_pages[next_sec])

        # Extract the section text using your jump-only extractor
        text, pages_used = extract_section_text(
            page_lookup=page_lookup,
            sec_heading=section_headings[sec_id],
            start_page=start_page,
            next_heading=next_heading,
            next_start_page=next_start_page,
            hard_end_page=hard_end_page,
        )

        chapter_id = int(sec_id.split(".")[0])

        section_docs.append(
            Document(
                page_content=text,
                metadata={
                    "chapter_id": chapter_id,
                    "section_id": sec_id,
                    "section_heading": section_headings[sec_id],
                    "start_page": start_page,
                    "hard_end_page": hard_end_page,
                    "pages_used": pages_used,
                    'source': 'psychology_book.pdf'
                },
            )
        )

    return section_docs

section_docs = build_section_documents_jump(section_pages, section_headings, page_lookup)

print(len(section_docs))
print(section_docs[0].metadata)          # should be 1.1
print(section_docs[1].metadata)          # should be 1.2
print(section_docs[0].page_content[:200])


88
{'chapter_id': 1, 'section_id': '1.1', 'section_heading': '1.1 What Is Psychology?', 'start_page': 20, 'hard_end_page': None, 'pages_used': [20, 21], 'source': 'psychology_book.pdf'}
{'chapter_id': 1, 'section_id': '1.2', 'section_heading': '1.2 History of Psychology', 'start_page': 21, 'hard_end_page': None, 'pages_used': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'source': 'psychology_book.pdf'}
1.1 What Is Psychology?
LEARNING OBJECTIVES
By the end of this section, you will be able to:
• Define psychology
• Understand the merits of an education in psychology
What is creativity? What are prej


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple
import hashlib

from langchain_text_splitters import RecursiveCharacterTextSplitter


def _stable_hash(text: str, length: int = 12) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()[:length]


# adds heading field in meta data

'''def _build_heading_prefix(meta: Dict[str, Any]) -> str:
    """
    Build a short heading prefix that will be INCLUDED in the chunk text
    so it affects embeddings (not just metadata).
    Adjust keys to match your metadata fields.
    """
    # Common metadata keys people use:
    # section_id: "1.2"
    # section_title: "Bias-Variance Tradeoff"
    # chapter_id: 1
    # chapter_title: "Linear Regression"
    section_id = meta.get("section_id")
    section_title = meta.get("section_heading") 
    chapter_id = meta.get("chapter_id") or meta.get("chapter") or ""
    chapter_title = meta.get("chapter_title") or meta.get("chapter_heading") or ""

    parts = []
    if chapter_id or chapter_title:
        ch = f"Chapter {chapter_id}".strip() if chapter_id else "Chapter"
        if chapter_title:
            parts.append(f"{ch}: {chapter_title}")
        else:
            parts.append(f"{ch}")
    if section_id or section_title:
        sec = f"Section {section_id}".strip() if section_id else "Section"
        if section_title:
            parts.append(f"{sec}: {section_title}")
        else:
            parts.append(f"{sec}")

    if not parts:
        return ""  # no prefix available
    return " | ".join(parts) + "\n\n"'''


def recursive_child_chunk(
    section_docs: List[Document],
    *, #key word arguments (no to positional arguments) 
    chunk_size: int = 1600,          # characters (not tokens)
    chunk_overlap: int = 200,        # characters
    separators: Optional[List[str]] = None,
    parent_id_key: str = "parent_section_id",
    section_id = "section_id",
) -> List[Document]:
    """
    Convert parent section-level docs -> child chunks using recursive splitting.

    Returns a flat list of child Documents suitable for vector DB ingestion.

    Assumptions:
    - Each parent Document represents one section (like 1.1, 1.2, ...)
    - Parent doc metadata contains something identifying the section (section_id)
    """

    if separators is None:
        # From "best" boundaries to "worst" (typical recursive strategy)
        separators = ["\n\n", "\n", ". ", " ", ""]

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators,
        length_function=len,
        add_start_index=True,  # lets us track start position in original text
    )

    child_docs: List[Document] = []

    for parent in section_docs:
        parent_text = parent.page_content or ""
        parent_meta = dict(parent.metadata or {})


        # Find section id in metadata
        section_id = str(parent_meta['section_id'])

        # Split and build child docs
        splits = splitter.create_documents(
            texts=[parent_text],
            metadatas=[parent_meta],
        )

        for i,d in enumerate(splits):
            meta = dict(d.metadata)
            meta[parent_id_key] = section_id
            meta['chunk_index'] = i

            start = meta.get('start_index')
            if isinstance(start,int):
                meta['start_char'] = start
                meta['end_char'] = start + len(d.page_content)

            # creating a stable chunk id
            meta['chunk_id'] = f"{section_id}::c{i:03d}::{_stable_hash(d.page_content)}"

            chunk_text = d.page_content or ""
            
            child_docs.append(Document(page_content = chunk_text, metadata = meta))

    return child_docs


# -------------------------
# Example usage:
# -------------------------
# child_docs = recursive_child_chunk(
#     section_docs=section_docs,
#     chunk_size=1800,
#     chunk_overlap=250,
#     add_heading_prefix_to_text=True,
# )
# print(len(child_docs), child_docs[0].metadata, child_docs[0].page_content[:200])


In [19]:
#so we have parent chunks per section (1.1,1.2,.....) which is good.
section_docs[4].page_content

"2.1 Why Is Research Important?\nLEARNING OBJECTIVES\nBy the end of this section, you will be able to:\n• Explain how scientific research addresses questions about behavior\n• Discuss how scientific research guides public policy\n• Appreciate how scientific research can be important in making personal decisions\nScientific research is a critical tool for successfully navigating our complex world. Without it, we would be\nforced to rely solely on intuition, other people’s authority, and blind luck. While many of us feel confident in\nour abilities to decipher and interact with the world around us, history is filled with examples of how very\nwrong we can be when we fail to recognize the need for evidence in supporting claims. At various times in\nhistory, we would have been certain that the sun revolved around a flat earth, that the earth’s continents did\nnot move, and that mental illness was caused by possession (Figure 2.2). It is through systematic scientific\nresearch that we dives

In [15]:
docs[0].metadata

{'producer': 'Prince 14.2 (www.princexml.com)',
 'creator': 'PyPDF',
 'creationdate': '2022-02-24T09:25:50-06:00',
 'moddate': '2022-03-01T11:18:04-06:00',
 'title': 'Psychology 2e',
 'source': 'psychology_book.pdf',
 'total_pages': 753,
 'page': 0,
 'page_label': '1'}

### DONOT EXECUTE BELOW CODE

In [ ]:
from typing import List, Tuple
from langchain_core.documents import Document
import copy

def page_label_int(d: Document) -> int:
    pl = d.metadata.get("page_label")
    if pl is None:
        return int(d.metadata.get("page", 0)) + 1
    return int(str(pl))

def filter_and_add_chapter(
    docs: List[Document],
    keep_ranges: List[Tuple[int, int]]
) -> List[Document]:

    out = []

    for d in docs:
        p = page_label_int(d)

        for idx, (a, b) in enumerate(keep_ranges):
            if a <= p <= b:
                # Create a copy so we don’t mutate original docs
                new_doc = Document(
                    page_content=d.page_content,
                    metadata=copy.deepcopy(d.metadata)
                )

                # Chapter number = index + 1
                new_doc.metadata["chapter_id"] = idx + 1

                out.append(new_doc)
                break

    return out





# ensuring every chapter starts in a page that contains 'chapter_number'.1
keep_ranges = [
    (19, 39),     # Chapter 1
    (48, 72),     # Chapter 2
    (84, 109),    # Chapter 3
    (122, 146),   # Chapter 4
    (158, 181),   # Chapter 5
    (194, 216),   # Chapter 6
    (226, 250),   # Chapter 7
    (260, 282),   # Chapter 8
    (292, 324),   # Chapter 9
    (334, 362),   # Chapter 10
    (372, 400),   # Chapter 11
    (412, 446),   # Chapter 12
    (460, 489),   # Chapter 13
    (498, 538),   # Chapter 14
    (550, 598),   # Chapter 15
    (612, 636),   # Chapter 16
    
]


filtered_docs = filter_and_add_chapter(docs, keep_ranges)

print(len(filtered_docs))
print(filtered_docs[21].metadata)




463
{'producer': 'Prince 14.2 (www.princexml.com)', 'creator': 'PyPDF', 'creationdate': '2022-02-24T09:25:50-06:00', 'moddate': '2022-03-01T11:18:04-06:00', 'title': 'Psychology 2e', 'source': 'psychology_book.pdf', 'total_pages': 753, 'page': 47, 'page_label': '48', 'chapter_id': 2}


In [51]:
print(filtered_docs[71].metadata)


{'producer': 'Prince 14.2 (www.princexml.com)', 'creator': 'PyPDF', 'creationdate': '2022-02-24T09:25:50-06:00', 'moddate': '2022-03-01T11:18:04-06:00', 'title': 'Psychology 2e', 'source': 'psychology_book.pdf', 'total_pages': 753, 'page': 108, 'page_label': '109', 'chapter_id': 3}


In [52]:
print(filtered_docs[72].page_content)




4.1 What Is Consciousness?
LEARNING OBJECTIVES
By the end of this section, you will be able to:
• Understand what is meant by consciousness
• Explain how circadian rhythms are involved in regulating the sleep-wake cycle, and how circadian cycles can be
disrupted
• Discuss the concept of sleep debt
Consciousness describes our awareness of internal and external stimuli. Awareness of internal stimuli
includes feeling pain, hunger, thirst, sleepiness, and being aware of our thoughts and emotions. Awareness of
external stimuli includes experiences such as seeing the light from the sun, feeling the warmth of a room, and
hearing the voice of a friend.
We experience different states of consciousness and different levels of awareness on a regular basis. We might
even describe consciousness as a continuum that ranges from full awareness to a deep sleep. Sleep is a state
marked by relatively low levels of physical activity and reduced sensory awareness that is distinct from periods
of rest that o

### Parent chunking based on Heading Number


1.1 What Is Psychology? 20
1.2 History of Psychology 21
1.3 Contemporary Psychology 30
1.4 Careers in Psychology 38 - 41
2.1 Why Is Research Important? 48
2.2 Approaches to Research 53
2.3 Analyzing Findings 60
2.4 Ethics 71 - 74
3.1 Human Genetics 84
3.2 Cells of the Nervous System 90
3.3 Parts of the Nervous System 96
3.4 The Brain and Spinal Cord 98
3.5 The Endocrine System 109 - 111
4.1 What Is Consciousness? 122
4.2 Sleep and Why We Sleep 126
4.3 Stages of Sleep 129
4.4 Sleep Problems and Disorders 133
4.5 Substance Use and Abuse 138
4.6 Other States of Consciousness 146 - 148
5.1 Sensation versus Perception 158
5.2 Waves and Wavelengths 161
5.3 Vision 165
5.4 Hearing 173
5.5 The Other Senses 176
5.6 Gestalt Principles of Perception 180 - 183
6.1 What Is Learning? 194
6.2 Classical Conditioning 195
6.3 Operant Conditioning 204
6.4 Observational Learning (Modeling) 215 - 216
7.1 What Is Cognition? 226
7.2 Language 230
7.3 Problem Solving 234
7.4 What Are Intelligence and Creativity? 240
7.5 Measures of Intelligence 243
7.6 The Source of Intelligence 249 - 252
8.1 How Memory Functions 260
8.2 Parts of the Brain Involved with Memory 267
8.3 Problems with Memory 271
8.4 Ways to Enhance Memory 281 - 284
9.1 What Is Lifespan Development? 292
9.2 Lifespan Theories 296
9.3 Stages of Development 304
9.4 Death and Dying 325 - 326
10.1 Motivation 334
10.2 Hunger and Eating 340
10.3 Sexual Behavior 346
10.4 Emotion 354 - 364
11.1 What Is Personality? 372
11.2 Freud and the Psychodynamic Perspective 374
11.3 Neo-Freudians: Adler, Erikson, Jung, and Horney 380
11.4 Learning Approaches 385
11.5 Humanistic Approaches 389
11.6 Biological Approaches 390
11.7 Trait Theorists 391
11.8 Cultural Understandings of Personality 396
11.9 Personality Assessment 398 - 402
12.1 What Is Social Psychology? 412
12.2 Self-presentation 418
12.3 Attitudes and Persuasion 421
12.4 Conformity, Compliance, and Obedience 427
12.5 Prejudice and Discrimination 434
12.6 Aggression 441
12.7 Prosocial Behavior 444 - 448
13.1 What Is Industrial and Organizational Psychology? 460
13.2 Industrial Psychology: Selecting and Evaluating Employees 468
13.3 Organizational Psychology: The Social Dimension of Work 479
13.4 Human Factors Psychology and Workplace Design 489 - 491
14.1 What Is Stress? 498
14.2 Stressors 508
14.3 Stress and Illness 514
14.4 Regulation of Stress 526
14.5 The Pursuit of Happiness 533 - 540
15.1 What Are Psychological Disorders? 550
15.2 Diagnosing and Classifying Psychological Disorders 554
15.3 Perspectives on Psychological Disorders 557
15.4 Anxiety Disorders 560
15.5 Obsessive-Compulsive and Related Disorders 566
15.6 Posttraumatic Stress Disorder 570
15.7 Mood and Related Disorders 572
15.8 Schizophrenia 582
15.9 Dissociative Disorders 586
15.10 Disorders in Childhood 588
15.11 Personality Disorders 594 - 600
16.1 Mental Health Treatment: Past and Present 612
16.2 Types of Treatment 617
16.3 Treatment Modalities 629
16.4 Substance-Related and Addictive Disorders: A Special Case 633
16.5 The Sociocultural Model and Therapy Utilization 635 - 638